In [1]:
import sys, os
sys.path.insert(0, '..')

In [2]:
import toolbox as tb

In [3]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [4]:
data_folder = '../data'

### Excercise 2.8

Load data

In [5]:
train = pd.read_table(os.path.join(data_folder, 'zip.train'), sep=' ', header=None)
test = pd.read_table(os.path.join(data_folder, 'zip.test'), sep=' ', header=None)

clean data objects

In [6]:
train[257].isnull().all()

True

In [7]:
train = train.drop(257, axis=1)

In [8]:
def filter_and_split(df):
    sub = df[0].isin([2,3])
    ret = df[sub].copy()
    features = ret.drop(0, axis=1)
    obj = (ret[0] == 2).astype(int) # 0,1 encoding
    return features, obj

In [9]:
def correctness_ratio(y_true, y_pred):
    correct = (y_true == y_pred).sum()
    return correct/len(y_true)

In [25]:
pred_errors = {}

In [26]:
X_train, Y_train = filter_and_split(train)
X_test, Y_test = filter_and_split(test)

In [27]:
lrmod = tb.linear_model(X_train, Y_train, add_constant=False, verbose=False)
y_train_pred = (lrmod.predict(X_train) > 0.5).astype(int)
y_test_pred = (lrmod.predict(X_test) > 0.5).astype(int)

In [29]:
pred_errors['lr'] = correctness_ratio(Y_train, y_train_pred), correctness_ratio(Y_test, y_test_pred)

In [31]:
for k in [1,3,5,7,15]:
    nnmod = KNeighborsClassifier(n_neighbors=k)
    nnmod.fit(X_train, Y_train)
    y_train_pred = nnmod.predict(X_train)
    y_test_pred = nnmod.predict(X_test)
    pred_errors[str(k)] = correctness_ratio(Y_train, y_train_pred), correctness_ratio(Y_test, y_test_pred)

In [35]:
pd.DataFrame(pred_errors, index=['train', 'test']).T

,train,test
lr,0.994240,0.958791
1,1.000000,0.975275
3,0.994960,0.969780
5,0.994240,0.969780
7,0.993521,0.967033
15,0.990641,0.961538
